In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks

os.chdir('./CycleSR/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [3]:
!pip install --upgrade pip
!pip uninstall scikit-image
!pip install scikit-image
import skimage
print(skimage.__version__)

Found existing installation: scikit-image 0.19.1
Uninstalling scikit-image-0.19.1:
  Would remove:
    /usr/local/bin/skivi
    /usr/local/lib/python3.7/dist-packages/doc/ext/*
    /usr/local/lib/python3.7/dist-packages/scikit_image-0.19.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_image.libs/libgomp-f7e03b3e.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/skimage/*
Proceed (Y/n)? n
0.19.1


In [4]:
# Prepare package
!pip install lpips
!pip install scikit-learn
if not os.path.exists("./pytorch_wavelets"):
  !git clone https://github.com/fbcotter/pytorch_wavelets
%cd ./pytorch_wavelets/
!pip install .
%cd ../

/content/drive/MyDrive/Colab Notebooks/CycleSR/pytorch_wavelets
Processing /content/drive/MyDrive/Colab Notebooks/CycleSR/pytorch_wavelets
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-wavelets: filename=pytorch_wavelets-1.3.0-py3-none-any.whl size=55125 sha256=abf4adbbee08879794e70ff9ae4c1a1afabb844591aec2ddfa15399c4d2268e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-19u7y1vh/wheels/c2/68/5e/51fa73a99f70fdfe25ce12e2bc3c4ece9a0b825a1a53184fe5
Successfully built pytorch-wavelets
  Attempting uninstall: pytorch-wavelets
    Found existing installation: pytorch-wavelets 1.3.0
    Uninstalling pytorch-wavelets-1.3.0:
      Successfully uninstalled pytorch-wavelets-1.3.0
/content/drive/MyDrive/Colab Notebooks/CycleSR


In [25]:
import glob
import random
import os
from PIL import Image
import numpy as np
import time
import datetime
import sys

import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.autograd import Variable

from tensorflow import summary
from torch.utils.tensorboard import SummaryWriter

import argparse
import itertools
import matplotlib.pyplot as plt

from pytorch_wavelets import DWTForward

import pdb
import skimage.metrics

from tqdm import tqdm

import lpips
loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
# loss_fn_vgg = lpips.LPIPS(net='vgg') # closer to "traditional" perceptual loss, when used for optimization

# import pytorch_fft.fft.autograd as fft
from utils import high_pass, low_pass

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.7/dist-packages/lpips/weights/v0.1/alex.pth


In [26]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('run/SR_baseline')

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [27]:
# Helper Functions
def tensor2image(tensor):
    image = 127.5*(tensor[0].cpu().float().numpy() + 1.0)
    if image.shape[0] == 1:
        image = np.tile(image, (3,1,1))
    return image.astype(np.uint8)


class ReplayBuffer():
    def __init__(self, max_size=50):
        assert (max_size > 0), 'Empty buffer or trying to create a black hole. Be careful.'
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0,1) > 0.5:
                    i = random.randint(0, self.max_size-1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))

class LambdaLR():
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert ((n_epochs - decay_start_epoch) > 0), "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch)/(self.n_epochs - self.decay_start_epoch)

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant(m.bias.data, 0.0)


In [28]:
# Dataloader
class ImageDataset(Dataset):
    def __init__(self, root, transforms_A=None, transforms_B=None, unaligned=False, mode='train'):
        self.transformA = transforms.Compose(transforms_A)
        self.transformB = transforms.Compose(transforms_B)

        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, 'trainA') + '/*.*'))
        self.files_B = sorted(glob.glob(os.path.join(root, 'trainB') + '/*.*'))

    def __getitem__(self, index):
        img_A = Image.open(self.files_A[index % len(self.files_A)]).convert('L')
        item_A = self.transformA(img_A)

        if self.unaligned:
            item_B = self.transformB(Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)]).convert('L'))
        else:
            item_B = self.transformB(Image.open(self.files_B[index % len(self.files_B)]).convert('L'))

        return {'A': item_A, 'B': item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

In [29]:
class Discriminator(nn.Module):
    """Defines a PatchGAN discriminator"""

    def __init__(self, input_nc=1, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d):
        """Construct a PatchGAN discriminator

        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            n_layers (int)  -- the number of conv layers in the discriminator
            norm_layer      -- normalization layer
        """
        super(Discriminator, self).__init__()
        use_bias = True
        kw = 4
        padw = 1
        sequence = [nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw), nn.LeakyReLU(0.2, True)]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):  # gradually increase the number of filters
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]  # output 1 channel prediction map
        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        """Standard forward."""
        return self.model(input)

In [30]:
def set_requires_grad(nets, requires_grad=False):
    """Set requies_grad=Fasle for all the networks to avoid unnecessary computations
    Parameters:
        nets (network list)   -- a list of networks
        requires_grad (bool)  -- whether the networks require gradients or not
    """
    if not isinstance(nets, list):
        nets = [nets]
    for net in nets:
        if net is not None:
            for param in net.parameters():
                param.requires_grad = requires_grad

In [31]:
def save_sample(epoch, tensor, suffix="_real"):
    output = tensor.cpu().detach().numpy().squeeze(0).squeeze(0)
    plt.imsave('./tmp/image_'+str(epoch+1)+suffix+'.jpeg', output, cmap="gray")

In [32]:
#### Defination of local variables
input_nc = 1
output_nc = 1
batchSize = 1
size_A, size_B = 256, 256
lr = 2e-4
n_epochs, epoch, decay_epoch = 100, 0, 10
n_cpu = 2
dataroot = "./dataset/OCTA_baseline"
cuda = True

In [33]:
if torch.cuda.is_available() and not cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

In [34]:
class UnetGenerator(nn.Module):
    """Create a Unet-based generator"""

    def __init__(self, input_nc=1, output_nc=1, num_downs=8, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet generator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            output_nc (int) -- the number of channels in output images
            num_downs (int) -- the number of downsamplings in UNet. For example, # if |num_downs| == 7,
                                image of size 128x128 will become of size 1x1 # at the bottleneck
            ngf (int)       -- the number of filters in the last conv layer
            norm_layer      -- normalization layer
        We construct the U-Net from the innermost layer to the outermost layer.
        It is a recursive process.
        """
        super(UnetGenerator, self).__init__()
        # construct unet structure
        unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=None, norm_layer=norm_layer, innermost=True)  # add the innermost layer
        for i in range(num_downs - 5):          # add intermediate layers with ngf * 8 filters
            unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer, use_dropout=use_dropout)
        # gradually reduce the number of filters from ngf * 8 to ngf
        unet_block = UnetSkipConnectionBlock(ngf * 4, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf * 2, ngf * 4, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf, ngf * 2, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        self.model = UnetSkipConnectionBlock(output_nc, ngf, input_nc=input_nc, submodule=unet_block, outermost=True, norm_layer=norm_layer)  # add the outermost layer

    def forward(self, input):
        """Standard forward"""
        hf = high_pass(input[0], i=5).unsqueeze(0).unsqueeze(0) # (1, 320) 5
        input = (hf+input)/2.0
        return self.model(input)


class UnetSkipConnectionBlock(nn.Module):
    """Defines the Unet submodule with skip connection.
        X -------------------identity----------------------
        |-- downsampling -- |submodule| -- upsampling --|
    """

    def __init__(self, outer_nc, inner_nc, input_nc=None,
                 submodule=None, outermost=False, innermost=False, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet submodule with skip connections.
        Parameters:
            outer_nc (int) -- the number of filters in the outer conv layer
            inner_nc (int) -- the number of filters in the inner conv layer
            input_nc (int) -- the number of channels in input images/features
            submodule (UnetSkipConnectionBlock) -- previously defined submodules
            outermost (bool)    -- if this module is the outermost module
            innermost (bool)    -- if this module is the innermost module
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers.
        """
        super(UnetSkipConnectionBlock, self).__init__()
        self.outermost = outermost
        use_bias = True
        if input_nc is None:
            input_nc = outer_nc
        downconv = nn.Conv2d(input_nc, inner_nc, kernel_size=4,
                             stride=2, padding=1, bias=use_bias)
        downrelu = nn.LeakyReLU(0.2, True)
        downnorm = norm_layer(inner_nc)
        uprelu = nn.ReLU(True)
        upnorm = norm_layer(outer_nc)

        if outermost:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1)
            down = [downconv]
            up = [uprelu, upconv, nn.Tanh()]
            model = down + [submodule] + up
        elif innermost:
            upconv = nn.ConvTranspose2d(inner_nc, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv]
            up = [uprelu, upconv, upnorm]
            model = down + up
        else:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv, downnorm]
            up = [uprelu, upconv, upnorm]

            if use_dropout:
                model = down + [submodule] + up + [nn.Dropout(0.5)]
            else:
                model = down + [submodule] + up

        self.model = nn.Sequential(*model)

    def forward(self, x):
        if self.outermost:
            return self.model(x)
        else:   # add skip connections
            return torch.cat([x, self.model(x)], 1)

In [35]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [36]:
###### Definition of variables ######
# Networks

netG_A2B = UnetGenerator()
netG_B2A = UnetGenerator()
netD_A = Discriminator()
netD_B = Discriminator()

if cuda:
    netG_A2B.cuda()
    netG_B2A.cuda()
    netD_A.cuda()
    netD_B.cuda()

netG_A2B.apply(weights_init_normal)
netG_B2A.apply(weights_init_normal)
netD_A.apply(weights_init_normal)
netD_B.apply(weights_init_normal)

# Lossess
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
# criterion_phase = phase_consistency_loss()
criterion_identity = torch.nn.L1Loss()


# Optimizers & LR schedulers
optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()), lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(itertools.chain(netD_A.parameters(), netD_B.parameters()), lr=lr, betas=(0.5, 0.999))


lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step)
lr_scheduler_D = torch.optim.lr_scheduler.LambdaLR(optimizer_D, lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step)

# Inputs & targets memory allocation
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
input_A = Tensor(batchSize, input_nc, size_A, size_A)
# input_B = Tensor(batchSize, output_nc, size_A, size_A)
input_B = Tensor(batchSize, output_nc, size_B, size_B)
target_real = Variable(Tensor(batchSize).fill_(1.0), requires_grad=False)
target_fake = Variable(Tensor(batchSize).fill_(0.0), requires_grad=False)

fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

# Dataset loader
transforms_A = [ 
                transforms.ToTensor(),
                # transforms.Normalize((0.246), (0.170)),
                transforms.Normalize((0.5), (0.5)),
                # transforms.CenterCrop(size_A),
                transforms.RandomCrop((size_A, size_A))
                ]
                
transforms_B = [ 
                transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5)),
                # transforms.Normalize((0.286), (0.200)),
                # transforms.CenterCrop(size_B),
                transforms.RandomCrop((size_B, size_B))
                ]
dataset = ImageDataset(dataroot, transforms_A=transforms_A, transforms_B=transforms_B, unaligned=True)
print (len(dataset))
dataloader = DataLoader(dataset, batch_size=batchSize, shuffle=True)

# Loss plot
# logger = Logger(n_epochs, len(dataloader))
###################################


354


In [37]:
import warnings
warnings.filterwarnings('ignore')

In [38]:
# lr_img = Image.open("./test/6x6_256/270_3.png").convert('L')
# hr_img = Image.open("./test/3x3_256/270_6.png").convert('L')

# T_1 = transforms.Compose([ transforms.ToTensor(),
#                 transforms.Normalize((0.5), (0.5)),
#                  ])
# T_2 = transforms.Compose([ transforms.ToTensor(),                         
#                 transforms.Normalize((0.5), (0.5))])

# lr_img = T_1(lr_img).cuda().unsqueeze(0)
# hr_img = T_2(hr_img).cuda().unsqueeze(0)

In [39]:
def eval(model):
  lr = "./dataset/test/6x6_256/"
  hr = "./dataset/test/3x3_256/"
  num, psnr, ssim, mse, nmi= 0, 0, 0, 0, 0
  T_1 = transforms.Compose([ transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5)),
                 ])
  T_2 = transforms.Compose([ transforms.ToTensor(),                         
                  transforms.Normalize((0.5), (0.5))])
  for i in tqdm(range(297)):
    lr_path = os.path.join(lr, str(i)+"_3.png")
    hr_path = os.path.join(hr, str(i)+"_6.png")
    if os.path.isfile(lr_path) and os.path.isfile(hr_path):
      lr_img = Image.open(lr_path).convert('L')
      hr_img = Image.open(hr_path).convert('L')
      
      lr_img = T_1(lr_img).cuda().unsqueeze(0)
      hr_img = T_2(hr_img).cuda().unsqueeze(0)
      
      sr_img = model(lr_img)

      yimg = sr_img.cpu().detach().numpy().squeeze(0).squeeze(0)
      gtimg = hr_img.cpu().detach().numpy().squeeze(0).squeeze(0)
      psnr += (skimage.metrics.peak_signal_noise_ratio(yimg, gtimg))
      ssim += (skimage.metrics.structural_similarity(yimg, gtimg))
      mse += (skimage.metrics.mean_squared_error(yimg, gtimg))
      nmi += (skimage.metrics.normalized_mutual_information(yimg, gtimg))
      num += 1
  print(" PSNR: %.4f SSIM: %.4f MSE: %.4f NMI: %.4f"%(psnr/num, ssim/num, mse/num, nmi/num))


In [40]:
# %tensorboard --logdir=run

In [ ]:
###### Training ######
globaliter = 0
for epoch in range(epoch, n_epochs):
    real_out, fake_out = None, None
    for i, batch in enumerate(dataloader):
        globaliter += 1
        real_A = Variable(input_A.copy_(batch['A']))
        real_B = Variable(input_B.copy_(batch['B']))

        ######### (1) forward #########
        fake_B = netG_A2B(real_A)
        recovered_A = netG_B2A(fake_B)
        fake_A = netG_B2A(real_B)
        recovered_B = netG_A2B(fake_A)


        ###### (2) G_A and G_B ######
        set_requires_grad([netD_A, netD_B], False)
        optimizer_G.zero_grad()

        pred_fake = netD_B(fake_B)
        loss_GAN_A2B = criterion_GAN(pred_fake, target_real)

        pred_fake = netD_A(fake_A)
        loss_GAN_B2A = criterion_GAN(pred_fake, target_real)

        idt_A = netG_A2B(real_B)
        loss_idt_B = criterion_identity(idt_A, real_B) * 0.5

        idt_B = netG_B2A(real_A)
        loss_idt_A = criterion_identity(idt_B, real_A) * 0.5
        

        loss_cycle_ABA = criterion_cycle(recovered_A, real_A)*10.0
        loss_cycle_BAB = criterion_cycle(recovered_B, real_B)*10.0

        loss_G = loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB + loss_idt_B + loss_idt_A

        loss_G.backward()        
        optimizer_G.step()

        ###### (3) D_A and D_B ######
        set_requires_grad([netD_A, netD_B], True)
        optimizer_D.zero_grad()

        # Real loss
        pred_real = netD_A(real_A)
        loss_D_real = criterion_GAN(pred_real, target_real)
        # Fake loss
        fake_A = fake_A_buffer.push_and_pop(fake_A)
        pred_fake = netD_A(fake_A.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)
        # Total loss
        loss_D_A = (loss_D_real + loss_D_fake)*0.5
        loss_D_A.backward()


        # Real loss
        pred_real = netD_B(real_B)
        loss_D_real = criterion_GAN(pred_real, target_real)      
        # Fake loss
        fake_B = fake_B_buffer.push_and_pop(fake_B)
        pred_fake = netD_B(fake_B.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)
        # Total loss
        loss_D_B = (loss_D_real + loss_D_fake)*0.5
        loss_D_B.backward()

        optimizer_D.step()

        writer.add_scalar('GAN A2B loss', loss_GAN_A2B, globaliter)
        writer.add_scalar('GAN B2A loss', loss_GAN_B2A, globaliter)
        writer.add_scalar('Cycle ABA loss', loss_cycle_ABA, globaliter)
        writer.add_scalar('Cycle BAB loss', loss_cycle_BAB, globaliter)
        writer.add_scalar('Identity A loss', loss_idt_A, globaliter)
        writer.add_scalar('Identity B loss', loss_idt_B, globaliter)
        writer.add_scalar('Dis A loss', loss_D_A, globaliter)
        writer.add_scalar('Dis B loss', loss_D_B, globaliter)
        
        ####################################
        ####################################

        if i == 1:
          x = real_A.detach()
          real_out = x
          fake_out = netG_A2B(x)
      
    save_sample(epoch, real_out, "_input")
    save_sample(epoch, fake_out, "_output")

    # Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D.step()



    # Save models checkpoints
    # # torch.save(LR_encoding.state_dict(), 'output/LR_encoding.pth')
    # if epoch%5==4:
    #   torch.save(netG_A2B.state_dict(), './baseline_output/netG_A2B_epoch'+str(epoch+1)+'.pth')
    print("Epoch (%d/%d) Finished" % (epoch+1, n_epochs))

    # sr_img = netG_A2B(lr_img)
    # LPIPS = loss_fn_alex(hr_img.cpu(), sr_img.cpu())

    # yimg = sr_img.cpu().detach().numpy().squeeze(0).squeeze(0)
    # hr_img_cpu = hr_img.cpu().detach().numpy().squeeze(0).squeeze(0)
    # psnr = skimage.metrics.peak_signal_noise_ratio(yimg, hr_img_cpu)
    # ssim = skimage.metrics.structural_similarity(yimg, hr_img_cpu)

    # print(("PSNR: %.4f SSIM: %.4f LPIPS:"%(psnr, ssim)), LPIPS.data)

    # if (epoch+1)%2 == 0:
    eval(netG_A2B)

Epoch (1/100) Finished


100%|██████████| 297/297 [06:04<00:00,  1.23s/it]


 PSNR: 14.1390 SSIM: 0.1693 MSE: 0.1550 NMI: 1.0295
Epoch (2/100) Finished


100%|██████████| 297/297 [01:05<00:00,  4.51it/s]


 PSNR: 15.4767 SSIM: 0.3265 MSE: 0.1140 NMI: 1.0425
Epoch (3/100) Finished


...

 PSNR: 16.7875 SSIM: 0.4327 MSE: 0.0852 NMI: 1.0496
Epoch (60/100) Finished


100%|██████████| 297/297 [01:04<00:00,  4.58it/s]


 PSNR: 16.7735 SSIM: 0.4278 MSE: 0.0853 NMI: 1.0491
Epoch (61/100) Finished


100%|██████████| 297/297 [01:05<00:00,  4.55it/s]


 PSNR: 17.1031 SSIM: 0.4429 MSE: 0.0793 NMI: 1.0536


In [ ]:
def result_save_sample(epoch, tensor=None, suffix="_real", img=None, img_mode=False):
    if tensor != None:
      output = tensor.cpu().detach().numpy().squeeze(0).squeeze(0)
      plt.imsave('./results/image_baseline_'+str(epoch)+suffix+'.jpeg', output, cmap="gray")
    if img_mode:
      plt.imsave('./results/image_baseline_'+str(epoch)+suffix+'.jpeg', img, cmap="gray")

In [ ]:
netG_A2B = torch.load('./baseline_output/netG_A2B_epoch100.pth')
type(netG_A2B)
model = UnetGenerator(output_nc, input_nc).cuda()
model.load_state_dict(netG_A2B, strict=False)

In [ ]:
img = dataset[0]['A']
x = img.unsqueeze(0).cuda()
plt.imshow(img.squeeze(0), "gray")
result_save_sample(1, tensor=x, suffix="_input")

In [ ]:
y = model(x)
yimg = y.cpu().detach().numpy().squeeze(0).squeeze(0)
plt.imshow(yimg, "gray")
result_save_sample(1, tensor=y, suffix="_output")

In [ ]:
import cv2
upsample = cv2.resize(img.squeeze(0).cpu().numpy(), dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
upsample.shape
result_save_sample(1, img_mode=True, img=upsample, suffix="_interpolation")

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
netG_A2B = torch.load('./baseline_output/netG_A2B_epoch100.pth')
type(netG_A2B)
model = UnetGenerator(output_nc, input_nc).cuda()
model.load_state_dict(netG_A2B, strict=False)
y = model(x)

# img = x.cpu().detach().numpy().squeeze(0).squeeze(0)
img = y.cpu().detach().numpy().squeeze(0).squeeze(0)
f = np.fft.fft2(img, axes=(-2, -1))
fshift = np.fft.fftshift(f)
res = np.log(np.abs(fshift))
pha = np.angle(fshift)
plt.figure(figsize=(11, 11))
plt.subplot(121), plt.imshow(img, 'gray')#, plt.title('Original Image')
plt.axis('off')
plt.subplot(122), plt.imshow(res, 'gray')#, plt.title('Fourier Amplitude')
plt.axis('off')
# plt.subplot(333), plt.imshow(pha, 'gray')#, plt.title('Fourier Phase')
# plt.axis('off')

In [ ]:
img = dataset[0]['A']
x = img.unsqueeze(0).cuda()
plt.imshow(img.squeeze(0), "gray")
plt.axis('off')
y = model(x)

In [ ]:
yimg = y.cpu().detach().numpy().squeeze(0).squeeze(0)
plt.imshow(yimg, "gray")
plt.axis('off')

In [ ]:
class Test_ImageDataset(Dataset):
    def __init__(self, root, transforms_test=None, unaligned=False, mode='test'):
        self.transformA = transforms.Compose(transforms_test)
        self.transformB = transforms.Compose(transforms_test)

        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, '6x6_256/') + '/*.*'))
        self.files_B = sorted(glob.glob(os.path.join(root, '3x3_256/') + '/*.*'))

    def __getitem__(self, index):
        img_A = Image.open(self.files_A[index % len(self.files_A)]).convert('L')
        print(self.files_A[index % len(self.files_A)])
        item_A = self.transformA(img_A)

        item_B = self.transformB(Image.open("./test/3x3_256/102_6.png").convert('L'))

        return {'A': item_A, 'B': item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

In [ ]:
test_path = "./test/"
transforms_test = [ 
                transforms.ToTensor(),
                # transforms.Normalize((0.246), (0.170)),
                transforms.Normalize((0.5), (0.5)) ]
test_dataset = Test_ImageDataset(test_path, transforms_test=transforms_test, unaligned=True)

In [ ]:
import cv2
img = test_dataset[2]['A']
gt = test_dataset[2]['B']
print(img.shape)
# img = cv2.resize(img.squeeze(0).cpu().numpy(), dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
x = torch.tensor(img).unsqueeze(0).cuda()
# plt.imshow(img.squeeze(0), "gray")
y = model(x)

In [ ]:
plt.imshow(img.squeeze(0), "gray")
plt.axis('off')

In [ ]:
plt.imshow(gt.squeeze(0), "gray")
plt.axis('off')

In [ ]:
yimg = y.cpu().detach().numpy().squeeze(0).squeeze(0)
plt.imshow(yimg, "gray")
plt.axis('off')

In [ ]:
f = np.fft.fft2(yimg, axes=(-2, -1))
fshift = np.fft.fftshift(f)
res = np.log(np.abs(fshift))
pha = np.angle(fshift)
plt.figure(figsize=(11, 11))
plt.subplot(331), plt.imshow(yimg, 'gray'), plt.title('Original Image')
plt.axis('off')
plt.subplot(332), plt.imshow(res, 'gray'), plt.title('Fourier Amplitude')
plt.axis('off')
plt.subplot(333), plt.imshow(pha, 'gray'), plt.title('Fourier Phase')
plt.axis('off')

Pick a pair of test data to evaluate

In [ ]:
netG_A2B = torch.load('./baseline_output/netG_A2B_epoch40.pth')
type(netG_A2B)
model = UnetGeneratorA2B(output_nc, input_nc).cuda()
model.load_state_dict(netG_A2B, strict=False)

In [ ]:
lr_img = Image.open("./test/6x6_256/270_3.png").convert('L')
hr_img = Image.open("./test/3x3_256/270_6.png").convert('L')
# lr_img = Image.open("./dataset/Colab_centered_OCTA/trainA/STDR403_20181029_101618_Angio (1)_R_001.png").convert('L')
# hr_img = Image.open("./dataset/Colab_centered_OCTA/trainB/STDR403_20181029_101802_Angio (1)_R_001.png").convert('L')
T_1 = transforms.Compose([ transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5)),
                transforms.Resize([128, 128]) ])
T_2 = transforms.Compose([ transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5))])
# lr_img = cv2.resize(np.array(lr_img), dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
# lr_img = torch.tensor(lr_img).unsqueeze(0).unsqueeze(0).cuda()
lr_img = T_1(lr_img).cuda().unsqueeze(0)
hr_img = T_2(hr_img).cuda().unsqueeze(0)
# lr_img.size()
sr_img = model(lr_img)

In [ ]:
ximg = lr_img.cpu().detach().numpy().squeeze(0).squeeze(0)
print(ximg.shape)
plt.imshow(ximg, "gray")

In [ ]:
yimg = sr_img.cpu().detach().numpy().squeeze(0).squeeze(0)
plt.imshow(yimg, "gray")

In [ ]:
gtimg = hr_img.cpu().detach().numpy().squeeze(0).squeeze(0)
plt.imshow(gtimg, "gray")

In [ ]:
import skimage.metrics
print(skimage.metrics.peak_signal_noise_ratio(yimg, gtimg))
print(skimage.metrics.structural_similarity(yimg, gtimg))

# Result

In [ ]:
import skimage.metrics
print(skimage.metrics.peak_signal_noise_ratio(yimg, gtimg))
print(skimage.metrics.structural_similarity(yimg, gtimg))

In [ ]:
lr_img = Image.open("./test/6x6_256/270_3.png").convert('L')
hr_img = Image.open("./test/3x3_256/270_6.png").convert('L')
lr_img = T_1(lr_img)
hr_img = T_2(hr_img)

In [ ]:
input = lr_img.cuda().unsqueeze(0)
output = model(input)
# output = output.cpu().detach().numpy().squeeze(0).squeeze(0)

In [ ]:
d = loss_fn_alex(hr_img.cpu(), output.cpu())
print(d)

In [ ]:
x = F.interpolate(input, scale_factor=2, mode='nearest')
d = loss_fn_alex(hr_img.cpu(), x.cpu())
print(d)